<a href="https://colab.research.google.com/github/Bisma-Shafiq/Crewai-Projects/blob/main/Automate_Event_Planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 google-generativeai langchain_google_genai

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [12]:
from crewai import Agent, Task, Crew

In [13]:
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
SERPER_API_KEY = userdata.get('SERPER_API_KEY')
serper_api_key=SERPER_API_KEY


In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [17]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",verbose=True,temperature=0.2,google_api_key=GOOGLE_API_KEY)

In [18]:
from crewai_tools import ScrapeWebsiteTool , SerperDevTool

In [23]:
scrape_tool = ScrapeWebsiteTool()
search_tool = SerperDevTool()

In [24]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=llm,
    allow_delegation=False
)

In [25]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=llm,
    allow_delegation=False
)

In [26]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=llm,
    allow_delegation=True
)

In [27]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [28]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [29]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [30]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [31]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True
)

In [32]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [33]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
Thought: I need to find a venue in San Francisco suitable for a Tech Innovation Conference.  I should search the internet for venues in San Francisco that can accommodate conferences and have the right technological infrastructure.

Action: Search the internet

Action Input: {"search_query": "conference venues San Francisco tech"}
 

I encountered an error while trying to use the tool. This was the error: 'SERPER_API_KEY'.
 Tool Search the internet accepts these inputs: Search the internet(search_query: 'string') - A tool that can be used to semantic search a query from a txt's content.

Thought: I need to find a venue in San Francisco suitable for a Tech Innovation Conference.  I'll try a different search query focusing on specific venue types known for tech events.

Action: Search the int

In [34]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [35]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Tech Innovation Conference: Marketing Activities and Attendee Engagement Report

**1. Event Overview:**

* **Event Name:** Tech Innovation Conference
* **Dates:** October 26th - 28th, 2024
* **Venue:** Moscone Center, San Francisco
* **Target Attendance:** 500+ attendees
* **Goal:** To effectively market the event and engage potential attendees to maximize participation.


**2. Marketing Strategy:**

**A. Pre-Event Marketing (July - October 2024):**

* **Website Development:** Create a dedicated event website with detailed information about the conference (agenda, speakers, registration, venue details, sponsorship opportunities).  The website will be optimized for search engines (SEO) and mobile devices.
* **Social Media Marketing:**  Launch a comprehensive social media campaign across platforms like Twitter, LinkedIn, Facebook, and potentially Instagram.  This will involve creating engaging content (videos, infographics, articles), running contests and giveaways, and utilizing relevant hashtags to increase visibility.  Regular updates will be posted to maintain audience engagement.
* **Email Marketing:** Develop an email marketing campaign targeting potential attendees based on their interests and professional backgrounds.  This will involve creating a series of emails announcing the event, highlighting key speakers and sessions, and promoting early bird registration discounts.
* **Public Relations:**  Issue press releases to relevant technology publications and media outlets to generate publicity for the conference.  We will also reach out to industry influencers and bloggers to promote the event.
* **Paid Advertising:**  Utilize paid advertising on social media platforms and search engines (Google Ads) to reach a wider audience.  The ads will target specific demographics and interests relevant to the conference.
* **Content Marketing:** Create valuable content related to the conference theme, such as blog posts, articles, and white papers, to attract potential attendees and establish thought leadership.  This content will be shared across various channels.
* **Partnerships:** Collaborate with relevant organizations and companies in the tech industry to cross-promote the event and reach their audiences.


**B. During-Event Marketing (October 26th - 28th, 2024):**

* **On-site Promotion:**  Set up engaging booths and displays at the Moscone Center to promote the event and encourage networking among attendees.
* **Social Media Engagement:**  Continue to actively engage with attendees on social media during the event, using live updates, photos, and videos to create excitement and encourage participation.
* **Networking Events:**  Organize networking events and social gatherings to facilitate connections among attendees and speakers.


**3. Attendee Engagement:**

* **Interactive Sessions:**  Incorporate interactive elements into the conference sessions, such as Q&A sessions, polls, and group discussions, to encourage active participation.
* **Networking Opportunities:**  Provide ample opportunities for attendees to network with each other and with speakers, such as dedicated networking breaks, social events, and online networking platforms.
* **Feedback Mechanisms:**  Gather feedback from attendees through surveys, feedback forms, and social media to improve future events.
* **Post-Event Follow-up:**  Send a thank-you email to attendees after the conference, including links to presentations, photos, and recordings.  We will also follow up on any leads generated during the event.


**4. Budget:**  A detailed budget will be developed once the venue costs are confirmed.  The budget will include costs for website development, marketing materials, advertising, speaker fees, venue rental, catering, and staff.


**5. Timeline:**  A detailed timeline will be developed once the venue is confirmed and the marketing activities are finalized.  The timeline will outline key milestones and deadlines for each marketing activity.


**6. Measurement and Evaluation:**

* **Website Analytics:**  Track website traffic, bounce rate, and conversion rates to measure the effectiveness of the website.
* **Social Media Analytics:**  Monitor social media engagement metrics, such as likes, shares, comments, and reach, to assess the success of the social media campaign.
* **Email Marketing Analytics:**  Track email open rates, click-through rates, and conversions to measure the effectiveness of the email marketing campaign.
* **Registration Numbers:**  Monitor registration numbers to track the overall success of the marketing efforts.
* **Post-Event Survey:**  Conduct a post-event survey to gather feedback from attendees and assess their overall satisfaction with the conference.


This report provides a framework for the marketing and attendee engagement strategy.  Specific details will be finalized once the venue contract is secured and the budget is approved.